In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
import json
import time
import re


In [ ]:
def df_lambda_replace_characters(df, column_name, new_column, dictionary):
    df[new_column] = df[column_name].apply(lambda x: replace_words(x, dictionary))       
    return df

In [ ]:
def replace_words(strtext, dictionary):
    pattern = re.compile(r'(?<!\w)(' + '|'.join(re.escape(key) for key in dictionary.keys()) + r')(?!\w)')
    result = pattern.sub(lambda x: dictionary[x.group()], strtext)
    return result

In [ ]:
with open("../replacement_dictionary_ordered.json") as json_file:
    replacement_dictionary_2 = json.load(json_file)

In [ ]:
states = ['TX', 'KS', 'AZ', 'TN', 'FL', 'GA', 'OK', 'UT', 'ME', 'CA', 'MS',
       'WA', 'OH', 'MA', 'AL', 'NC', 'CO', 'MO', 'KY', 'NE', 'NY',
       'VA', 'MD', 'HI', 'WV', 'MN', 'AK', 'IN', 'PA', 'SC', 'LA', 'NJ',
       'RI', 'WY']

In [ ]:
state = states[0]
noun_file = 'new_avm_df__full_data_state_'+state+'_noun_sentences.fea'
noun_sentence_df_T_ri_filtered = pd.read_feather(noun_file)

noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered[noun_sentence_df_T_ri_filtered['count'] >\
                                                                noun_sentence_df_T_ri_filtered['count'].max()*0.005]

noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered.loc[:,['noun_sentence', 'count',
                                                                      'AVM_Error_Indicators']]

In [ ]:
len(noun_sentence_df_T_ri_filtered)

In [ ]:
column = 'AVM_Error_Indicators'
nelements = (len(noun_sentence_df_T_ri_filtered[column].values[0]))
list_tmp = [str(i) for i in range(nelements)] 
column_names = [column + '_' + str(i) for i in list_tmp] 
print(column_names)
split_df = pd.DataFrame(noun_sentence_df_T_ri_filtered[column].tolist(), columns=column_names, 
                        index=noun_sentence_df_T_ri_filtered.index)
noun_sentence_df_T_ri_filtered = pd.concat([noun_sentence_df_T_ri_filtered, split_df], axis=1)
noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered.drop(columns = [column])

noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered.loc[:,['noun_sentence','count',
                                                                       'AVM_Error_Indicators_0','AVM_Error_Indicators_1']]
noun_sentence_df_T_ri_filtered = df_lambda_replace_characters(noun_sentence_df_T_ri_filtered,  'noun_sentence', 
                                                   'noun_sentence_corrections', replacement_dictionary_2)
noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered.loc[:,['noun_sentence_corrections','count',
                                                                       'AVM_Error_Indicators_0','AVM_Error_Indicators_1']]
noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered.sort_values(by=['noun_sentence_corrections'])

noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered.add_suffix('_'+state)


In [ ]:
noun_sentence_df_T_ri_filtered[noun_sentence_df_T_ri_filtered['noun_sentence_corrections_TX']=='stainless steel appliances']

In [ ]:
noun_sentence_df_T_ri_filtered_full = noun_sentence_df_T_ri_filtered.copy(deep=True).\
                                rename(columns={'noun_sentence_corrections_'+state:'noun_sentence'})

In [ ]:
repeated = np.unique(np.array(list(noun_sentence_df_T_ri_filtered_full[noun_sentence_df_T_ri_filtered_full['noun_sentence'].\
                                         duplicated()]['noun_sentence'].values)))

In [ ]:
for rrr in repeated:
    #print(rrr)
    tmp = noun_sentence_df_T_ri_filtered_full[noun_sentence_df_T_ri_filtered_full['noun_sentence']==rrr]
    multi_mean  = ((tmp['count_'+state]*tmp['AVM_Error_Indicators_0_'+state]).sum())/tmp['count_'+state].sum()
    first_part_num = ((tmp['count_'+state]*tmp['AVM_Error_Indicators_1_'+state]).sum())
    second_part_num = ((tmp['count_'+state]*(tmp['AVM_Error_Indicators_0_'+state]-multi_mean)**2.).sum())
    multi_variance = (first_part_num + second_part_num) / tmp['count_'+state].sum()
    noun_sentence_df_T_ri_filtered_full['count_'+state]\
            [noun_sentence_df_T_ri_filtered_full['noun_sentence']==rrr] = tmp['count_'+state].sum()
    noun_sentence_df_T_ri_filtered_full['AVM_Error_Indicators_0_'+state]\
            [noun_sentence_df_T_ri_filtered_full['noun_sentence']==rrr] = multi_mean
    noun_sentence_df_T_ri_filtered_full['AVM_Error_Indicators_1_'+state]\
            [noun_sentence_df_T_ri_filtered_full['noun_sentence']==rrr] = multi_variance
    
    #print(tmp)
    #print('MM',multi_mean)
    #print('MV',multi_variance)
    #print('count',tmp['count_'+state].sum())
    #print(noun_sentence_df_T_ri_filtered_full[noun_sentence_df_T_ri_filtered_full['noun_sentence']==rrr])
    

In [ ]:
noun_sentence_df_T_ri_filtered_full = noun_sentence_df_T_ri_filtered_full.drop_duplicates(subset='noun_sentence')

In [ ]:
noun_sentence_df_T_ri_filtered_full = noun_sentence_df_T_ri_filtered_full.set_index('noun_sentence')

In [ ]:
noun_sentence_df_T_ri_filtered_full.sort_values('count_TX')

In [ ]:
noun_sentence_df_T_ri_filtered_full.loc['fridge stays']

In [ ]:
for li in noun_sentence_df_T_ri_filtered_full.index[2900:]:
    print(li)

In [ ]:
import math

In [ ]:
for state in states[1:]:
    print('full ',len(noun_sentence_df_T_ri_filtered_full))
    tic = time.time()
    print(state)
    noun_file = 'new_avm_df__full_data_state_'+state+'_noun_sentences.fea'
    noun_sentence_df_T_ri_filtered = pd.read_feather(noun_file)
    
    noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered[noun_sentence_df_T_ri_filtered['count'] >\
                                                                    math.ceil(noun_sentence_df_T_ri_filtered['count'].max()*0.005)]

    print(len(noun_sentence_df_T_ri_filtered))
    
    if len(noun_sentence_df_T_ri_filtered)>=1:
        noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered.loc[:,['noun_sentence', 'count',
                                                                              'AVM_Error_Indicators']]

        column = 'AVM_Error_Indicators'
        nelements = (len(noun_sentence_df_T_ri_filtered[column].values[0]))
        list_tmp = [str(i) for i in range(nelements)] 
        column_names = [column + '_' + str(i) for i in list_tmp] 
        #print(column_names)
        split_df = pd.DataFrame(noun_sentence_df_T_ri_filtered[column].tolist(), columns=column_names, 
                                index=noun_sentence_df_T_ri_filtered.index)
        noun_sentence_df_T_ri_filtered = pd.concat([noun_sentence_df_T_ri_filtered, split_df], axis=1)
        noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered.drop(columns = [column])

        noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered.loc[:,['noun_sentence','count',
                                                                               'AVM_Error_Indicators_0','AVM_Error_Indicators_1']]
        noun_sentence_df_T_ri_filtered = df_lambda_replace_characters(noun_sentence_df_T_ri_filtered,  'noun_sentence', 
                                                           'noun_sentence_corrections', replacement_dictionary_2)
        noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered.loc[:,['noun_sentence_corrections','count',
                                                                               'AVM_Error_Indicators_0','AVM_Error_Indicators_1']]
        noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered.sort_values(by=['noun_sentence_corrections'])
        
        noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered.add_suffix('_'+state)
        
        noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered.\
                                    rename(columns={'noun_sentence_corrections_'+state:'noun_sentence'})
        
        repeated = np.unique(np.array(list(noun_sentence_df_T_ri_filtered\
                                           [noun_sentence_df_T_ri_filtered['noun_sentence'].\
                                           duplicated()]['noun_sentence'].values)))
        
        for rrr in repeated:
            #print(rrr)
            tmp = noun_sentence_df_T_ri_filtered[noun_sentence_df_T_ri_filtered['noun_sentence']==rrr]
            multi_mean  = ((tmp['count_'+state]*tmp['AVM_Error_Indicators_0_'+state]).sum())/tmp['count_'+state].sum()
            first_part_num = ((tmp['count_'+state]*tmp['AVM_Error_Indicators_1_'+state]).sum())
            second_part_num = ((tmp['count_'+state]*(tmp['AVM_Error_Indicators_0_'+state]-multi_mean)**2.).sum())
            multi_variance = (first_part_num + second_part_num) / tmp['count_'+state].sum()
            noun_sentence_df_T_ri_filtered['count_'+state]\
                    [noun_sentence_df_T_ri_filtered['noun_sentence']==rrr] = tmp['count_'+state].sum()
            noun_sentence_df_T_ri_filtered['AVM_Error_Indicators_0_'+state]\
                    [noun_sentence_df_T_ri_filtered['noun_sentence']==rrr] = multi_mean
            noun_sentence_df_T_ri_filtered['AVM_Error_Indicators_1_'+state]\
                    [noun_sentence_df_T_ri_filtered['noun_sentence']==rrr] = multi_variance


        noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered.drop_duplicates(subset='noun_sentence')
        noun_sentence_df_T_ri_filtered = noun_sentence_df_T_ri_filtered.set_index('noun_sentence')

        print('filtered ', len(noun_sentence_df_T_ri_filtered))
        print('added ', len(noun_sentence_df_T_ri_filtered)+len(noun_sentence_df_T_ri_filtered_full))
        noun_sentence_df_T_ri_filtered_full = noun_sentence_df_T_ri_filtered_full.merge( noun_sentence_df_T_ri_filtered,
                                                                                    how='outer',left_index=True, right_index=True)
    
        
    print('full ',len(noun_sentence_df_T_ri_filtered_full))
    
    toc = time.time()
    print(toc - tic)
    print('')

In [ ]:
len(noun_sentence_df_T_ri_filtered_full)

In [ ]:
noun_sentence_df_T_ri_filtered_full

In [ ]:
noun_sentence_df_T_ri_filtered_full.head(20)

In [ ]:
noun_sentence_df_T_ri_filtered_full.head(20)

In [ ]:
len(noun_sentence_df_T_ri_filtered_full)

In [ ]:
file = 'noun_phrases_by_state_new_avm'

In [ ]:
noun_sentence_df_T_ri_filtered_full.to_csv(file+'.csv')
noun_sentence_df_T_ri_filtered_full.reset_index().to_feather(file+'.fea')

In [ ]:
noun_sentence_df_T_ri_filtered_full

In [ ]:
noun_prhases_new_data = pd.DataFrame(columns = ['noun_sentence', 'state', 'count', 'mean', 'variance'])

In [ ]:
for stateg in states:
    #print(noun_sentence_df_T_ri_filtered_full.columns)
    print(stateg)
    res = [i for i in list(noun_sentence_df_T_ri_filtered_full.columns) if stateg in i]
    print(res)
    if len(res)>0:
        tmp = noun_sentence_df_T_ri_filtered_full.loc[:,['count_'+stateg, 'AVM_Error_Indicators_0_'+stateg, 'AVM_Error_Indicators_1_'+stateg]]
        tmp = tmp.rename(columns={'count_'+stateg:'count',
                                  'AVM_Error_Indicators_0_'+stateg:'mean',
                                  'AVM_Error_Indicators_1_'+stateg:'variance'})
        tmp['state'] = stateg
        tmp = tmp.reset_index()
        noun_prhases_new_data = noun_prhases_new_data.append(tmp,ignore_index=True)

In [ ]:
noun_prhases_new_data

In [ ]:
noun_prhases_new_data.to_csv('noun_phrases_new_data.csv')

In [ ]:
noun_sentence_df_T_ri_filtered_full.loc[:,['count_'+stateg, 'AVM_Error_Indicators_0_'+stateg, 'AVM_Error_Indicators_1_'+stateg]]

In [ ]:
stateg='TX'

In [ ]:
['count_'+stateg, 'AVM_Error_Indicators_0_'+stateg, 'AVM_Error_Indicators_1_'+stateg]

In [ ]:

import tensorflow as tf
import tensorflow_hub as hub

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"

model = hub.load(module_url)
print ("module %s loaded" % module_url)


In [ ]:
def embed(input):
    return model(input)

In [ ]:
noun_sentence_df_T_ri_filtered_full_embeddings = noun_sentence_df_T_ri_filtered_full.reset_index()

In [ ]:
noun_sentence_df_T_ri_filtered_full_embeddings

In [ ]:
column = 'noun_sentence'
tic = time.time()
new_column = column + '_embeddings'
noun_sentence_df_T_ri_filtered_full_embeddings[new_column] = noun_sentence_df_T_ri_filtered_full_embeddings[column].apply(lambda x: embed([x]).numpy()[0]) 
toc = time.time()
print(toc - tic)

In [ ]:
noun_sentence_df_T_ri_filtered_full_embeddings

In [ ]:
column = 'noun_sentence_embeddings'
nelements = (len(noun_sentence_df_T_ri_filtered_full_embeddings[column].values[0]))
list_tmp = [str(i) for i in range(nelements)] 
column_names = [column + '_' + str(i) for i in list_tmp] 
print(column_names)
split_df = pd.DataFrame(noun_sentence_df_T_ri_filtered_full_embeddings[column].tolist(), columns=column_names, 
                        index=noun_sentence_df_T_ri_filtered_full_embeddings.noun_sentence)

In [ ]:
split_df.T

In [ ]:
correlations_df = split_df.T.corr(method='pearson')

In [ ]:
correlations_df

In [ ]:
correlations_df.index

In [ ]:
list_of_lists = []
corr_threshold = 0.82

In [ ]:
for idd,sentence in enumerate(correlations_df.index):
    print(idd)
    #print(correlations_df.index[correlations_df.iloc[idd,:]>corr_threshold])
    list_of_lists.append(list(correlations_df.index[correlations_df.iloc[idd,:]>corr_threshold]))

In [ ]:
len(list_of_lists)

In [ ]:
noun_prhase_map = pd.DataFrame(columns = ['noun_phrase', 'maps_to'])

In [ ]:
countc = 0
for idd,list_id in enumerate(list_of_lists):
    flag = len(noun_prhase_map[noun_prhase_map['noun_phrase']==list_id[0]])
    print(idd, flag, countc)
    print(list_id)
    if flag==0:
        countc += len(list_id)
        for element in list_id:
            noun_prhase_map = noun_prhase_map.append({'noun_phrase' : element, 'maps_to' : list_id[0]},  ignore_index = True)

In [ ]:
noun_prhase_map

In [ ]:
len(noun_prhase_map[noun_prhase_map['noun_phrase']==list_id[0]])

In [ ]:
len(noun_prhase_map[noun_prhase_map['noun_phrase']=='casa'])

In [ ]:
noun_prhase_map.to_csv('noun_phrase_map.csv')
noun_prhase_map.reset_index().to_feather('noun_phrase_map.fea')

In [ ]:
correlations_df_groups = correlations_df.copy(deep=True)

In [ ]:
groups = []
len_groups = []

In [ ]:
while len(correlations_df_groups) > 0:
    idd = correlations_df_groups.index[0]
    print(idd)
    print(len(correlations_df_groups))
    group = list(np.array(correlations_df_groups.loc[idd][correlations_df_groups.loc[idd]>0.4].index))
    groups.append(group)
    len_groups.append(len(group))
    correlations_df_groups = correlations_df_groups.drop(labels = group, axis = 1)
    correlations_df_groups = correlations_df_groups.drop(labels = group, axis = 0)

In [ ]:
len(groups)
np.array(len_groups).max()

In [ ]:
order_groups = np.array(len_groups).argsort()[::-1]

In [ ]:
groups[order_groups[0]]

In [ ]:
len(groups)

In [ ]:
noun_prhase_groups = pd.DataFrame(columns = ['group','noun_phrase'])

In [ ]:
for idd, pos in enumerate(order_groups):
    list_g = groups[pos]
    print(idd+1, ' ',len(list_g))
    for element in list_g:
        noun_prhase_groups = noun_prhase_groups.append({'group' : idd+1, 'noun_phrase' : element},  ignore_index = True)

In [ ]:
noun_prhase_groups

In [ ]:
noun_prhase_groups.to_csv('noun_phrase_groups.csv')
noun_prhase_groups.reset_index().to_feather('noun_phrase_groups.fea')

In [ ]:
from wordcloud import WordCloud

In [ ]:
wc = WordCloud(background_color="white", colormap="Dark2", max_font_size=150, random_state=42)

In [ ]:
plt.rcParams['figure.figsize']=[20,10]

In [ ]:
for idd, pos in enumerate(order_groups):
    list_g = groups[pos]
    print(idd+1, ' ',len(list_g))
    wc.generate(' '.join(list_g))
    plt.imshow(wc,interpolation="bilinear")
    plt.axis('off')
    plt.show()

In [ ]:
noun_sentence_df_T_ri_filtered

In [ ]:
for list_g in groups[0:1]:
    print(len(list_g))
    wc.generate(' '.join(list_g))
    plt.imshow(wc,interpolation="bilinear")
    plt.axis('off')
    plt.show()